In [61]:
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
import subprocess
import random
import chess
from time import time

In [26]:
df = pd.read_csv("stockfish_evals.csv")
df = df[df.columns[2:]]
temp_list = []
for x in range(10, 48):
    temp_list.append('fen' + str(x))
df = df.reindex(columns = df.columns.tolist() + temp_list)
a = subprocess.Popen("./stockfish", cwd = "/home/ksasagaw/Chess/Stockfish/src", shell = True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
a.stdin.write('uci\n'.encode())
a.stdin.flush()
a.stdin.write('isready\n'.encode())
a.stdin.flush()
for x in range(1,30):
    a.stdout.readline() #read every set up line

for x in df[df.columns[:37]]:
    for idx, moves in df[x].iteritems():
        if(str(moves) == 'nan'):
            break
        a.stdin.write(('position startpos moves ' + str(moves) + '\n').encode())
        a.stdin.flush()
        a.stdin.write('d\n'.encode())
        a.stdin.flush()
        for y in range (0,23):
            b = str(a.stdout.readline())
            if(y == 20):
                c = b.split('Fen: ')[1].replace('\\n', '').replace('\'','')
        df.loc[idx, 'fen' + x.replace('uniq move', '')] = c


In [129]:
piece_to_list = {
            "p":0,
            "n":1,
            "b":2,
            "r":3,
            "q":4,
            "k":5,
            "P":6,
            "N":7,
            "B":8,
            "R":9,
            "Q":10,
            "K":11,
            ".":12
        }
def toTensor(board):
    v_board = []
    for pc in str(board).split():
        if pc=="\n":
            continue
        else:
            v_board.append(piece_to_list[pc])
    return torch.Tensor(v_board).reshape(1,-1)

In [130]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.l1 = nn.Linear(64, 48)
        self.l2 = nn.Linear(48, 12)
        self.l3 = nn.Linear(12, 1)

    def forward(self, v):
        v = torch.relu(self.l1(v))
        v = torch.relu(self.l2(v))
        v = self.l3(v)
        return v
    

In [131]:
def getRandTrainingPair(df):
    row = random.randint(0, len(df.index) - 1)
    move = random.randint(10, 47)
    if(str(df.loc[row, ('uniq move' + str(move) + '_score')]) == 'nan' or str(df.loc[row, ('fen' + str(move))]) == 'nan'):
        return getRandTrainingPair(df)
    return torch.as_tensor(df.loc[row, ('uniq move' + str(move) + '_score')]/100, dtype=torch.float), df.loc[row, ('fen' + str(move))]


In [132]:
model = Model()

criterion = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)



for epoch in range(1, 2000):
    running_loss = 0
    i = 0
    for iters in range(1, 3000):
        start = time()
        running_loss = 0
        optimizer.zero_grad()
        score, fen = getRandTrainingPair(df)
        #score = torch.as_tensor(-500, dtype=torch.float)
        score = torch.unsqueeze(score,-1)
        score = torch.unsqueeze(score,0)
        board = chess.Board(fen)
        vec_tensor = toTensor(board)
        output =  model(vec_tensor)
        #print(output, score, epoch)

        loss = criterion(output, score)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(loss)
        running_loss += loss.item()
        i += 1
    if epoch%1==0:
        print('Epoch %d loss: %.3f' % (epoch, running_loss))
        print('Time for this epoch: {}'.format(time()-start))

Epoch 1 loss: 1.398
Time for this epoch: 0.0009684562683105469
Epoch 2 loss: 2.132
Time for this epoch: 0.0011191368103027344
Epoch 3 loss: 0.668
Time for this epoch: 0.0010578632354736328
Epoch 4 loss: 2.908
Time for this epoch: 0.0010638236999511719
Epoch 5 loss: 8.028
Time for this epoch: 0.0012912750244140625
Epoch 6 loss: 3.282
Time for this epoch: 0.00180816650390625
Epoch 7 loss: 1.912
Time for this epoch: 0.0010421276092529297
Epoch 8 loss: 3.182
Time for this epoch: 0.0017974376678466797
Epoch 9 loss: 0.138
Time for this epoch: 0.0010535717010498047
Epoch 10 loss: 0.248
Time for this epoch: 0.002227306365966797
Epoch 11 loss: 0.922
Time for this epoch: 0.0010254383087158203
Epoch 12 loss: 0.898
Time for this epoch: 0.0009899139404296875
Epoch 13 loss: 1.042
Time for this epoch: 0.0017888545989990234
Epoch 14 loss: 0.098
Time for this epoch: 0.0010466575622558594
Epoch 15 loss: 5.834
Time for this epoch: 0.0009098052978515625
Epoch 16 loss: 4.427
Time for this epoch: 0.00083565

KeyboardInterrupt: 

In [211]:
df = pd.read_csv('games.csv')
df = df[df['turns'] > 30]
df = df[df['turns'] < 70]
df = df[df['rated'] == True]
df = df[df['victory_status'] == 'mate']
df = df[['moves','turns', 'winner']]

move_list = df['moves']
temp_list = []
for x in range(10, df['turns'].max()):
    temp_list.append('move' + str(x))
df = df.reindex(columns = df.columns.tolist() + temp_list)
print(df)

for idx, moves in move_list.iteritems():
    count = 10
    for y in range(30, len(moves.split())):
        name = 'move' + str(count)
        mveList = ''
        for x in range(0,count):
            if(x == 0):
                mveList += moves.split()[x]
            else:
                mveList += ' ' + moves.split()[x]
        df.loc[idx, name] = mveList
        
        count += 1
        
'''temp_list = []
for x in range(10, df['turns'].max()):
    temp_list.append('uniq move' + str(x))
df = df.reindex(columns = df.columns.tolist() + temp_list)
for x in range(10, int(df['turns'].max())):
    name = 'move' + str(x)
    unq_name = 'uniq move' + str(x)
    df[unq_name] = pd.Series(df[name].unique()).dropna()'''
    
df = df.dropna(how = 'all')
df

                                                   moves  turns winner  \
2      e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...     61  white   
3      d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...     61  white   
25     d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd...     69  white   
27     d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4 a3 B...     54  black   
29     d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4 Bxe...     66  black   
...                                                  ...    ...    ...   
20044  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf...     37  white   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 B...     36  black   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...     43  white   
20051  e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...     58  black   
20055  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...     35  white   

       move10  move11  move12  move13  move14  move15  move16  ...  move59  \
2         NaN     NaN     NaN    

moves  turns winner  \
2      e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...     61  white   
3      d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...     61  white   
25     d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd...     69  white   
27     d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4 a3 B...     54  black   
29     d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4 Bxe...     66  black   
...                                                  ...    ...    ...   
20044  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf...     37  white   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 B...     36  black   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...     43  white   
20051  e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...     58  black   
20055  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...     35  white   

                                      move10  \
2           e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5   
3       d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6   
25       d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7   
27        d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6   
29        d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6   
...                                      ...   
20044      e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4   
20051    e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5   
20055     d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7   

                                          move11  \
2            e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4   
3       d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2   
25      d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+   
27         d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3   
29        d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3   
...                                          ...   
20044      e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O   
20049   e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3   
20051     e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3   
20055     d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3   

                                               move12  \
2              e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5   
3        d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7   
25        d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6   
27          d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4   
29         d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4   
...                                               ...   
20044      e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7   
20046    e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+   
20051       e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5   
20055      d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O   

                                                  move13  \
2            e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5   
3       d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O-O   
25       d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3   
27          d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4 a3   
29       d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4 Bxe4   
...                                                  ...   
20044    e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf6   
20046   e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 Bg5   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...   
20051     e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5   
20055    d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd2   

                                                  move14  \
2        e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6   
3      d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...   
25     d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd...   
27     d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4 a3 B...   
29     d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4 Bxe...   
...                                                  ...   
20044  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf...   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 B...   
20049  e4 e6 N

In [ ]:
for x in df:
    for idx, moves in df[x].iteritems():
        if(x == 'turns' or x == 'winner' or x == 'moves'):
            break
        if(str(moves) == 'nan'):
            print('pass')
            continue
        temp_moves = str(moves) + " \*"
        subprocess.run("echo " + temp_moves + " > ~/Chess/temp_pgn.txt", shell = True, cwd  = "/home/ksasagaw/Chess", capture_output=True)
        pgn = subprocess.run("./pgn-extract -Wuci ~/Chess/temp_pgn.txt", shell = True, cwd  = "/home/ksasagaw/Chess/pgn-extract", capture_output=True)
        pgn = str(pgn.stdout).split('\\n')[8]
        print(pgn)
        pgn = pgn.replace('*','')
        df.at[idx,x] = pgn
    print(x)
df

moves
turns
winner
e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a7a5 *
d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b8c6 *
d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f8e7 *
d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e7e6 *
d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f8d6 *
e2e4 c7c5 g1f3 d7d6 f1b5 c8d7 b5d7 b8d7 e1g1 g8f6 *
e2e4 e7e6 f1c4 d7d5 e4d5 e6d5 c4b3 d8e7 g1e2 g8f6 *
d2d4 d7d6 c1f4 e7e6 g1f3 b8c6 e2e3 g8f6 b1d2 d6d5 *
e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 d4c5 f8c5 g1f3 b8c6 *
e2e4 e7e6 f2f4 d7d5 e4e5 c7c5 g1f3 b8c6 c2c3 g8h6 *
d2d4 g8f6 c1f4 d7d5 g1f3 b8c6 e2e3 c8g4 h2h3 g4h5 *
a2a4 e7e5 h2h4 d7d5 b2b3 b8c6 e2e3 c8f5 f1b5 g8e7 *
e2e4 e7e6 g1f3 d7d5 e4d5 e6d5 d2d4 c8g4 f1e2 d8e7 *
d2d4 g8f6 g1f3 b7b6 c2c3 c8b7 d1b3 e7e6 f3e5 b8c6 *
e2e4 e7e5 g1f3 b8c6 f1b5 f8c5 b5c6 d7c6 h2h3 g8f6 *
e2e4 e7e5 f1c4 g8f6 d2d4 e5d4 d1d4 b8c6 d4e3 f8b4 *
d2d4 d7d5 g1f3 g8f6 c1f4 b8c6 e2e3 c8f5 f1b5 e7e6 *
e2e4 c7c5 g1f3 g8f6 d2d4 f6e4 d4c5 e4c5 d1d5 e7e6 *
e2e4 e7e5 g1f3 b8c6 f1c4 g8f6 f3g5 d7d5 e4d5 

e2e4 e7e5 g1f3 b8c6 f1c4 g8f6 d2d4 e5d4 e1g1 f6e4 *
e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 c2c3 b8c6 f1b5 d8b6 *
e2e4 e7e5 g1f3 b8c6 f1c4 c6d4 f3d4 e5d4 d1h5 d8f6 *
e2e4 e7e5 g1f3 b8c6 f1c4 h7h6 b1c3 g8f6 d2d3 f8c5 *
e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 c2c3 b8c6 f1b5 c8d7 *
e2e4 c7c5 g1f3 b8c6 d2d4 c5d4 f3d4 e7e5 d4b5 d7d6 *
e2e4 e7e5 g1f3 b8c6 b1c3 f8b4 a2a3 b4c3 d2c3 d7d6 *
d2d4 g8f6 b1c3 d7d5 e2e3 c8f5 f1d3 f5d3 d1d3 e7e6 *
e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 c2c3 b8c6 g1f3 c8d7 *
c2c4 e7e5 b1c3 g8f6 g1f3 b8c6 g2g3 d7d5 c4d5 f6d5 *
g1f3 d7d6 g2g3 e7e5 d2d3 b8c6 f1g2 g8f6 b1d2 d6d5 *
d2d4 g8f6 g2g3 g7g6 f1g2 f8g7 c2c3 e8g8 b1d2 d7d6 *
e2e4 e7e5 g1f3 b8c6 f1b5 g8e7 e1g1 a7a6 b5c6 e7c6 *
d2d4 d7d5 g1f3 c7c5 b1d2 e7e6 e2e3 b8c6 f1d3 f8d6 *
d2d4 d7d5 g1f3 b8c6 b1d2 e7e6 e2e3 g8f6 f1d3 f8d6 *
e2e4 e7e6 d2d4 d7d5 b1c3 c7c5 g1f3 a7a6 f1d3 b8c6 *
e2e4 e7e5 g1f3 b8c6 b1c3 g8f6 d2d3 f8b4 c1d2 e8g8 *
e2e4 e7e5 g1f3 c7c6 f3e5 d8e7 d2d4 c6c5 c2c3 c5d4 *
d2d4 d7d5 g2g3 g7g6 b2b3 f8g7 c1b2 g8f6 f1g2 e8g8 *
e2e4 c7c5 g1

g1f3 b8c6 g2g3 b7b6 f1h3 c8b7 h3g4 c6e5 f3e5 b7h1 *
e2e4 e7e5 d1h5 b8c6 f1c4 g8h6 d2d4 d8f6 d4e5 c6e5 *
e2e4 e7e5 d1h5 d7d6 f1c4 g8h6 d2d4 e5d4 c1h6 d8e7 *
d2d4 b8c6 c2c4 e7e5 d4d5 c6d4 b1c3 d7d6 f2f4 c8f5 *
e2e4 e7e5 d1h5 g8f6 h5e5 f8e7 d2d4 b8c6 e5f4 c6d4 *
e2e4 e7e5 g1f3 b8c6 b1c3 d7d6 f1c4 c8g4 h2h3 g4f3 *
e2e4 e7e5 g1f3 d7d5 f3e5 d5e4 f1c4 g8h6 d2d3 d8f6 *
e2e4 e7e5 g1f3 f8c5 f3e5 d8h4 g2g3 h4e4 d1e2 e4h1 *
e2e4 b8c6 d2d4 e7e5 d4d5 c6d4 c2c3 f8c5 c3d4 e5d4 *
e2e4 d7d5 e4e5 b8c6 d2d4 f7f6 g1f3 c8g4 h2h3 g4f3 *
e2e4 g7g6 d1f3 f8g7 f1c4 g8h6 d2d4 e8g8 f3h3 d7d5 *
e2e4 e7e5 d1f3 g8f6 g1e2 d7d5 e4d5 e5e4 f3b3 f6d5 *
e2e4 e7e5 d1h5 b8c6 f1c4 g7g6 h5f3 g8f6 d2d3 f8c5 *
e2e4 e7e5 g1f3 c7c5 f3e5 d7d6 e5f3 g8f6 d2d4 f6e4 *
e2e4 e7e6 e4e5 c7c6 g1f3 g8h6 d2d4 b8a6 f3g5 d8a5 *
e2e4 e7e5 f1c4 d8h4 g1f3 h4f6 d2d4 d7d5 d4e5 f6b6 *
e2e4 e7e5 g1f3 f8c5 b1c3 b8c6 c3a4 d8e7 a4c5 e7c5 *
e2e4 e7e5 g1f3 d8f6 f1c4 a7a5 a2a4 c7c6 e1g1 b7b5 *
d2d4 e7e6 c2c4 d8f6 g1f3 f8b4 c1d2 b4d6 e2e4 d6f4 *
g1f3 d7d5 d2

e2e4 c7c5 g1f3 a7a6 d2d4 c5d4 f3d4 e7e5 d4b3 g8f6 *
e2e4 b8c6 g1f3 e7e5 d2d4 g7g6 d4d5 c6d4 c2c4 d4f3 *
e2e4 e7e5 g1f3 b8c6 d2d4 e5d4 f3d4 c6d4 d1d4 g8f6 *
e2e4 d7d5 b1c3 d5d4 f1b5 c7c6 b5c6 b8c6 c3b5 e7e5 *
e2e4 e7e5 d2d4 d7d5 e4d5 e5d4 d5d6 d8d6 g1f3 b8c6 *
d2d4 d7d5 c2c4 g7g6 g1f3 b8c6 a2a4 d5c4 e2e3 e7e5 *
e2e4 c7c5 d2d4 b8a6 c2c3 e7e6 g1f3 f8d6 f1e2 g8f6 *
e2e4 e7e5 f1c4 f8c5 g1f3 b8c6 b1c3 d7d6 e1g1 f7f5 *
e2e4 e7e5 f2f4 f7f6 f1c4 f8c5 g1e2 b8c6 c2c3 d7d6 *
e2e4 e7e5 d1h5 b8c6 f1c4 g7g6 h5f3 g8f6 h2h3 f8g7 *
c2c4 e7e5 a2a3 b8c6 b1c3 g8f6 e2e4 f8c5 h2h3 d7d6 *
e2e4 d7d5 e4d5 d8d5 b1c3 d5e5 d1e2 e5c5 g1f3 c8f5 *
d2d4 d7d5 g1f3 c8f5 c2c4 e7e6 b1c3 b8c6 c4d5 e6d5 *
e2e4 e7e6 g1f3 d7d5 e4d5 e6d5 d2d4 g8f6 f1d3 f8d6 *
d2d4 d7d5 c1f4 c8f5 e2e3 e7e6 c2c4 f8b4 b1c3 b4c3 *
d2d4 d7d5 c1f4 g8f6 e2e3 c8f5 g1f3 h7h6 c2c3 e7e6 *
e2e4 e7e5 g1f3 d7d6 d2d4 f7f6 b1c3 c8e6 d4d5 e6g4 *
e2e3 e7e5 f1b5 a7a6 b5c4 d7d6 d1e2 b8c6 e2h5 g7g6 *
e2e4 b7b6 d2d4 c8a6 f1a6 b8a6 g1f3 c7c6 d1e2 d8c7 *
d2d4 b7b6 b1

e2e4 c7c6 c2c3 d7d5 f2f3 g8f6 f1d3 e7e5 d1b3 c8e6 *
e2e4 e7e5 b1c3 g8f6 g2g4 d7d6 g4g5 c8g4 f2f3 g4e6 *
e2e4 c7c6 g1f3 d7d5 e4d5 c6d5 d2d4 e7e6 b1c3 b8c6 *
e2e4 c7c6 g1f3 d7d5 d2d3 g8f6 c1f4 b8d7 e4e5 f6g4 *
e2e4 e7e5 f2f4 d8h4 g2g3 h4f6 d2d3 e5f4 g3f4 d7d5 *
e2e4 c7c6 d2d4 d7d5 b1c3 d5e4 c3e4 c8f5 e4g5 d8d5 *
e2e4 e7e5 f1b5 c7c6 b5a4 d7d5 e4d5 d8d5 g1f3 d5e4 *
e2e4 e7e5 g1f3 b8c6 d2d3 g8f6 c1g5 h7h6 g5f6 g7f6 *
e2e4 c7c5 d2d4 c5d4 d1d4 b8c6 d4c5 e7e5 g1f3 f8c5 *
e2e4 e7e5 f2f4 e5f4 g1f3 g8f6 d2d3 d7d5 e4e5 f6d7 *
d2d4 g8f6 c2c4 e7e6 b1c3 d7d5 e2e3 f8b4 g1f3 c7c5 *
e2e4 e7e6 e4e5 d7d6 e5d6 d8d6 d2d4 g7g6 c2c3 f8g7 *
e2e4 e7e5 d2d4 e5d4 d1d4 b8c6 d4a4 d7d6 c2c3 c8d7 *
c2c4 e7e5 e2e4 d8f6 f2f3 f8c5 g1e2 d7d6 e2c3 c7c6 *
e2e4 e7e5 f1c4 d8h4 d1e2 f8c5 g2g3 h4f6 g1f3 b8c6 *
e2e4 e7e5 d1f3 g8h6 f1c4 d8e7 d2d3 d7d6 c1h6 g7h6 *
e2e4 e7e5 d1f3 g8f6 f1c4 d7d6 f3b3 d8e7 d2d3 b7b6 *
g1f3 d7d5 f3d4 e7e5 d4b5 c8d7 b5c7 d8c7 b1a3 d5d4 *
b1c3 d7d5 c3d5 d8d5 d2d3 b8c6 e2e4 d5d4 c2c3 d4f6 *
g1f3 g7g6 f3

...

e2e4 e7e5 g1f3 f7f6 d2d4 d7d6 d4e5 d6e5 f1d3 c8g4 e1g1 d8d7 c1e3 a7a6 b2b4 f8b4 *
c2c4 e7e5 b1c3 g8f6 e2e4 d7d5 c4d5 f8c5 h2h3 e8g8 f1e2 c7c6 d5c6 d8d4 d2d3 d4f2 *
e2e4 e7e5 g1f3 b8c6 f1b5 f8c5 e1g1 d7d6 h2h3 c8d7 c2c3 a7a6 b5a4 b7b5 a4b3 c6b8 *
e2e4 c7c5 f1c4 d7d6 d2d3 g8f6 g1f3 a7a6 e1g1 b7b5 c4f7 e8f7 b1c3 c8b7 c1f4 e7e5 *
e2e4 e7e6 d2d4 b7b6 g1f3 c8b7 f1d3 g8f6 e4e5 f6g8 c2c4 d7d6 e5d6 f8d6 e1g1 d8f6 *
e2e4 b8c6 g1f3 e7e5 f1b5 d7d6 e1g1 g8f6 f1e1 c8d7 c2c3 a7a6 b5c4 b7b5 c4b3 f8e7 *
e2e4 c7c5 f1c4 d7d6 a2a4 g8f6 d2d3 e7e6 g1f3 h7h6 e1g1 d6d5 e4d5 e6d5 f1e1 f8e7 *
e2e4 e7e6 d2d4 d7d5 e4e5 c7c5 c2c3 b8c6 g1f3 c8d7 f1e2 a8c8 e1g1 c5d4 c3d4 c6b4 *
e2e4 c7c5 f1c4 d7d6 b1c3 g8f6 g1f3 a7a6 d2d4 c5d4 f3d4 e7e5 d4f3 b7b5 c4d5 a8a7 *
e2e4 e7e5 g1f3 b8c6 f1b5 g8f6 e1g1 f8d6 f1e1 a7a6 b5a4 b7b5 a4b3 c6a5 c2c3 a5b3 *
e2e4 c7c5 f1c4 b8c6 d1h5 e7e6 f2f4 g8f6 h5f3 d7d6 d2d3 c6d4 f3d1 b7b5 c4b3 d4b3 *
e2e4 e7e5 g1f3 d7d6 f1c4 d8e7 e1g1 b8c6 c2c3 g8f6 f1e1 c8e6 c4b5 e6d7 d2d4 a7a6 *
e2e4 d7d6 d2d4 g

d2d4 e7e6 c2c4 g8f6 b1c3 c7c5 d4d5 e6d5 c4d5 d7d6 e2e4 f8e7 g2g3 e8g8 f1g2 c8g4 g1f3 *
e2e4 d7d5 d2d3 e7e6 g1f3 c7c6 b1c3 g8f6 c1d2 d8b6 f1e2 b6b2 d1b1 b2b1 a1b1 b8a6 c3a4 *
e2e4 c7c5 g1f3 e7e6 d2d4 c5d4 f3d4 a7a6 b1c3 b7b5 c1e3 c8b7 f2f3 d8c7 d1d2 g8f6 e1c1 *
e2e4 e7e5 g1f3 b8c6 f1b5 d7d6 d2d4 c8d7 e1g1 g8f6 f1e1 a7a6 b5c6 d7c6 b1c3 f8e7 c1g5 *
d2d4 d7d5 c2c4 e7e6 g1f3 g8f6 b1c3 a7a6 c1g5 f8e7 e2e3 h7h6 g5f6 e7f6 c4d5 e6d5 d1b3 *
e2e4 c7c5 g1f3 b8c6 b1c3 g7g6 d2d4 c5d4 f3d4 f8g7 c1e3 e7e6 f1c4 g8e7 e1g1 e8g8 d1d2 *
e2e4 c7c5 g1f3 b8c6 d2d4 c5d4 f3d4 g8f6 b1c3 e7e5 d4c6 b7c6 c1d2 a8b8 d1c1 f8c5 f1e2 *
pass
e2e4 c7c5 d2d4 c5d4 d1d4 b8c6 d4e3 g7g6 b1c3 f8g7 h2h3 g8f6 g2g4 d7d5 e4d5 f6d5 c3d5 *
d2d4 d7d5 c2c4 g8f6 c4d5 f6d5 e2e4 d5b6 c1e3 e7e6 b1c3 g7g6 f1b5 c8d7 b5d7 b8d7 g1f3 *
pass
e2e4 c7c5 e4e5 b8c6 g1f3 f7f6 e5f6 g8f6 f1c4 d7d5 c4b5 a7a6 b5c6 b7c6 d2d4 d8a5 c2c3 *
b2b3 e7e5 c1b2 b8c6 g1f3 d7d6 d2d4 e5d4 f3d4 c6d4 b2d4 d8h4 d4b2 g8f6 g2g3 h4b4 c2c3 *
d2d4 c7c6 c2c4 d7d5 e2e3 g8f6 f1d

e2e4 e7e5 g1f3 b8c6 f1c4 d7d6 f3g5 d8g5 d2d4 c6d4 c1g5 d4e6 c4e6 c8e6 e1g1 h7h6 g5h4 *
pass
e2e4 e7e5 g1f3 d7d5 d2d4 e5d4 f3d4 d5e4 b1c3 f8c5 c1e3 b8c6 f1b5 a7a6 b5c6 b7c6 e1g1 *
e2e4 e7e5 g1f3 d7d6 b1c3 a7a6 f1c4 h7h6 e1g1 b7b5 c4b3 c7c5 a2a4 c5c4 b3a2 b5b4 c3d5 *
d2d4 d7d5 b1c3 g8f6 h2h3 b8c6 g1f3 c8f5 c1f4 d8d7 e2e3 c6b4 f1e2 b4c2 e1f1 c2a1 d1a1 *
e2e4 e7e5 d2d3 g8f6 g1f3 d7d5 f3e5 d5e4 c1f4 e4d3 f1d3 f8d6 d1e2 e8g8 e1g1 d6e5 f4e5 *
e2e4 e7e6 g1f3 d7d6 b1c3 g8f6 d2d4 g7g6 c1g5 f8g7 e4e5 d6e5 d4e5 h7h6 e5f6 g7f6 g5f6 *
d2d4 d7d5 e2e4 d5e4 b1c3 g8f6 f2f3 e4f3 g1f3 b8c6 f1d3 e7e5 d4e5 f6g4 h2h3 g4e5 f3e5 *
e2e4 c7c5 f1c4 e7e6 g1f3 b8c6 b1c3 a7a6 e1g1 b7b5 c4b3 f8e7 f1e1 d8c7 d2d4 c5c4 d4d5 *
e2e4 c7c5 g1f3 b8c6 d2d4 c5d4 f3d4 e7e5 d4c6 b7c6 f1c4 g8f6 b1c3 f8e7 e1g1 e8g8 c1g5 *
e2e4 e7e5 b1c3 g8f6 f1c4 d7d6 d2d3 b8c6 h2h3 a7a6 f2f4 b7b5 c4b3 b5b4 c3d5 a6a5 a2a3 *
pass
e2e4 c7c5 c2c4 e7e5 b1c3 g7g5 d2d3 h7h5 g1f3 f7f6 h2h3 b8c6 g2g3 d7d6 f1g2 c6d4 f3d4 *
e2e4 e7e5 g1f3 b8c6 f1b5 a7a6 b5c

f2f4 d7d5 g1f3 b8c6 d2d4 g8f6 b1c3 c8g4 e2e3 e7e6 f1b5 f8d6 h2h3 g4f5 b2b3 f5e4 c1b2 *
e2e4 d7d5 b1c3 d5e4 c3e4 b8c6 g1f3 e7e5 f1b5 c8d7 e1g1 a7a6 b5a4 g8f6 e4f6 d8f6 a4c6 *
g2g3 g7g6 f1g2 f8g7 g1f3 b8c6 b1c3 d7d6 e1g1 g8h6 d2d3 e7e5 h2h3 d8f6 c1h6 g7h6 e2e3 *
g2g3 b7b6 f1g2 b8c6 g1f3 c8b7 d2d4 g7g6 d4d5 c6b4 e2e4 b4a6 e1g1 e7e5 f3e5 d8f6 c1f4 *
pass
pass
e2e4 d7d6 g1f3 g8f6 f1c4 g7g6 d2d3 f8g7 c1g5 e8g8 d1d2 c7c6 g5h6 b8d7 g2g4 f6g4 h6g7 *
d2d4 g8f6 c2c4 d7d5 b1c3 d5c4 g1f3 b8c6 e2e3 c8g4 f1c4 g4f3 d1f3 e7e6 e1g1 f8d6 f1d1 *
d2d4 g8f6 c2c4 b8c6 b1c3 d7d5 c1f4 a7a6 e2e3 g7g6 g1f3 f6h5 f4g3 h5g3 h2g3 d5c4 f1c4 *
d2d4 e7e6 c2c4 g8f6 b1c3 b7b6 c1f4 c8b7 g1f3 f8b4 e2e3 b4c3 b2c3 d7d6 h2h3 f6h5 f4h2 *
d2d4 g8f6 c2c4 d7d6 b1c3 c7c6 e2e4 e7e5 d4d5 b8d7 f2f3 f8e7 c1e3 e8g8 d1d2 d7b6 g2g4 *
d2d4 c7c5 d4c5 b8c6 g1f3 d8a5 c1d2 a5c5 e2e3 d7d5 a2a3 g8f6 b2b4 c5d6 c2c4 d5d4 c4c5 *
d2d4 e7e6 c2c4 c7c6 d4d5 c6d5 c4d5 e6d5 d1d5 b8c6 b1c3 d8a5 d5e4 g8e7 c1d2 f7f5 e4e3 *
d2d4 d7d5 c2c4 b8c6 b1c3 e7e6 g1f

e2e4 b8c6 d2d4 b7b6 d4d5 c6b8 b1c3 g7g6 g1f3 f8g7 c1e3 d7d6 e3d4 g7f8 d4h8 c8b7 f1b5 *
e2e4 b7b6 d2d4 b8c6 d4d5 c6e5 c1f4 e5g4 h2h3 g4f6 b1c3 g7g6 c3b5 c8a6 b5c7 d8c7 f4c7 *
d2d4 d7d5 c2c4 d5c4 g1f3 b7b5 a2a4 c7c6 a4b5 c6b5 b1c3 b8c6 c3b5 d8b6 b5c3 b6b4 e2e3 *
e2e4 e7e5 g1f3 b8c6 f1c4 h7h6 f3e5 c6e5 d2d4 e5c4 c1e3 c4e3 f2f4 e3d1 b1c3 d1c3 a1b1 *
pass
d2d4 d7d5 g1f3 b8c6 c2c4 d5c4 b1c3 g8f6 c1f4 g7g6 c3b5 e7e5 f3e5 c6e5 f4e5 f8d6 b5d6 *
e2e4 e7e5 g1f3 b8c6 b1c3 d7d6 f1b5 c8d7 d2d3 f8e7 e1g1 a7a6 b5a4 h7h5 b2b3 g7g5 c1b2 *
g1f3 g8f6 c2c4 b8c6 d2d4 f6e4 e2e3 e7e5 f1d3 e4f6 b1c3 f8b4 e1g1 e5d4 e3d4 d7d6 f1e1 *
pass
d2d4 d7d5 g1f3 b8c6 c1f4 e7e6 e2e3 h7h6 f1b5 c8d7 e1g1 g7g5 f4e5 f7f6 e5g3 h6h5 h2h3 *
pass
pass
e2e4 e7e5 b1c3 f8c5 d1e2 d7d6 g1f3 h7h6 d2d3 g8e7 c3d5 e8g8 f3g5 e7d5 b2b3 h6g5 e2h5 *
e2e4 e7e5 g1f3 d7d6 f1c4 b8c6 d2d3 g8f6 f3g5 f6e4 g5f7 d8h4 g2g3 h4f6 d3e4 c8h3 d1h5 *
pass
e2e4 e7e5 g1f3 b8c6 d2d3 h7h6 b1c3 g8f6 f1e2 c6d4 f3d4 e5d4 c3a4 f8d6 e2f3 e8g8 c2c4 *
pass
e2e4 e7e5 g1f

In [ ]:
temp_list = []
for x in range(10, 59):
    temp_list.append('fen' + str(x))
df = df.reindex(columns = df.columns.tolist() + temp_list)
a = subprocess.Popen("./stockfish", cwd = "/home/ksasagaw/Chess/Stockfish/src", shell = True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
a.stdin.write('uci\n'.encode())
a.stdin.flush()
a.stdin.write('isready\n'.encode())
a.stdin.flush()
for x in range(1,30):
    a.stdout.readline() #read every set up line

for x in df[df.columns[:37]]:
    for idx, moves in df[x].iteritems():
        if(str(moves) == 'nan'):
            break
        a.stdin.write(('position startpos moves ' + str(moves) + '\n').encode())
        a.stdin.flush()
        a.stdin.write('d\n'.encode())
        a.stdin.flush()
        for y in range (0,23):
            b = str(a.stdout.readline())
            if(y == 20):
                c = b.split('Fen: ')[1].replace('\\n', '').replace('\'','')
        df.loc[idx, 'fen' + x.replace('uniq move', '')] = c

In [210]:
df

moves  turns winner  \
2      e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...     61    NaN   
3      d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...     61    NaN   
25     d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd...     69    NaN   
27     d4 d5 Nc3 Nf6 Nf3 Nc6 e3 Bf5 h4 e6 g3 Bb4 a3 B...     54    NaN   
29     d4 d5 c4 Nf6 Nc3 Nc6 Nf3 e6 e3 Bd6 Bd3 Ne4 Bxe...     66    NaN   
...                                                  ...    ...    ...   
20044  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf...     37    NaN   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 B...     36    NaN   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...     43    NaN   
20051  e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...     58    NaN   
20055  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...     35    NaN   

                                                  move10  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044  e2e4 e7e6 d2d4 d7d5 e4e5 b8c6 g1f3 f7f6 f1b5 c...   
20046  e2e4 d7d6 d2d4 g8f6 e4e5 d6e5 f1e2 e5d4 g1f3 b...   
20049  e2e4 e7e6 g1f3 d7d5 b1c3 f8b4 e4d5 e6d5 d2d4 c...   
20051  e2e4 e7e6 g1f3 d7d5 f1b5 c8d7 c2c4 c7c6 b5a4 d...   
20055  d2d4 d7d5 c1f4 b8c6 e2e3 g8f6 c2c3 e7e6 g1f3 f...   

                                                  move11  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044  e2e4 e7e6 d2d4 d7d5 e4e5 b8c6 g1f3 f7f6 f1b5 c...   
20046  e2e4 d7d6 d2d4 g8f6 e4e5 d6e5 f1e2 e5d4 g1f3 b...   
20049  e2e4 e7e6 g1f3 d7d5 b1c3 f8b4 e4d5 e6d5 d2d4 c...   
20051  e2e4 e7e6 g1f3 d7d5 f1b5 c8d7 c2c4 c7c6 b5a4 d...   
20055  d2d4 d7d5 c1f4 b8c6 e2e3 g8f6 c2c3 e7e6 g1f3 f...   

                                                  move12  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044  e2e4 e7e6 d2d4 d7d5 e4e5 b8c6 g1f3 f7f6 f1b5 c...   
20046  e2e4 d7d6 d2d4 g8f6 e4e5 d6e5 f1e2 e5d4 g1f3 b...   
20049  e2e4 e7e6 g1f3 d7d5 b1c3 f8b4 e4d5 e6d5 d2d4 c...   
20051  e2e4 e7e6 g1f3 d7d5 f1b5 c8d7 c2c4 c7c6 b5a4 d...   
20055  d2d4 d7d5 c1f4 b8c6 e2e3 g8f6 c2c3 e7e6 g1f3 f...   

                                                  move13  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044  e2e4 e7e6 d2d4 d7d5 e4e5 b8c6 g1f3 f7f6 f1b5 c...   
20046  e2e4 d7d6 d2d4 g8f6 e4e5 d6e5 f1e2 e5d4 g1f3 b...   
20049  e2e4 e7e6 g1f3 d7d5 b1c3 f8b4 e4d5 e6d5 d2d4 c...   
20051  e2e4 e7e6 g1f3 d7d5 f1b5 c8d7 c2c4 c7c6 b5a4 d...   
20055  d2d4 d7d5 c1f4 b8c6 e2e3 g8f6 c2c3 e7e6 g1f3 f...   

                                                  move14  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e

In [ ]:
for idx, x in df['winner'].iteritems():
    if x == 'white':
        df.loc[idx, 'winner'] = 0
    elif x == 'black':
        df.loc[idx, 'winner'] = 1
    else:
        df.loc[idx, 'winner'] = np.nan
        


In [ ]:
class Model2(torch.nn.Module):
    
    def __init__(self):
        super(Model2, self).__init__()
        
        self.l1 = nn.Linear(64, 48)
        self.l2 = nn.Linear(48, 12)
        self.l3 = nn.Linear(12, 1)

    def forward(self, v):
        v = torch.relu(self.l1(v))
        v = torch.relu(self.l2(v))
        v = torch.sigmoid(self.l3(v))
        return v

In [ ]:
model2 = Model2()

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

df1 = df.reset_index()

for epoch in range(1, 30):
    running_loss = 0
    i = 0
    for x in df.iterrows():
        for y in range(10, 68):
            print(x)
            start = time()
            running_loss = 0
            optimizer.zero_grad()
            score = torch.as_tensor(x['winner'], dtype=torch.float)
            #score = torch.as_tensor(-500, dtype=torch.float)
            score = torch.unsqueeze(score,-1)
            score = torch.unsqueeze(score,0)
            fen = x[y]
            board = chess.Board(fen)
            vec_tensor = toTensor(board)
            output = model(vec_tensor)
            #print(output, score, epoch)

            loss = criterion(output, score)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print(loss)
            running_loss += loss.item()
            i += 1
    if epoch%1==0:
        print('Epoch %d loss: %.3f' % (epoch, running_loss))
        print('Time for this epoch: {}'.format(time()-start))

In [189]:
def getRandTrainingPair2(df):
    row = random.randint(0, df.shape[0] - 1)
    move = random.randint(10, 47)
    while(str(df.loc[row, ('fen' + str(move))]) == 'nan'):
        row = random.randint(0, df.shape[0] - 1)
        move = random.randint(10, 47)
    return torch.as_tensor(df.loc[row, 'winner'], dtype=torch.float), df.loc[row, ('fen' + str(move))]

In [200]:
df

moves  turns winner  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...     61    NaN   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...     61    NaN   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...     69    NaN   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...     54    NaN   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...     66    NaN   
...                                                  ...    ...    ...   
20044  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf...     37    NaN   
20046  e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 B...     36    NaN   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...     43    NaN   
20051  e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...     58    NaN   
20055  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...     35    NaN   

                                                  move10  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044                  e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7   
20046              e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6   
20049              e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4   
20051                e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5   
20055                 d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7   

                                                  move11  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044              e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O   
20046          e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O   
20049           e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3   
20051             e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3   
20055             d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3   

                                                  move12  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044         e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7   
20046       e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5   
20049     e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+   
20051          e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5   
20055         d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O   

                                                  move13  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e2e3 c8f5 h2h4 e...   
29     d2d4 d7d5 c2c4 g8f6 b1c3 b8c6 g1f3 e7e6 e2e3 f...   
...                                                  ...   
20044    e4 e6 d4 d5 e5 Nc6 Nf3 f6 Bb5 Bd7 O-O Nge7 exf6   
20046   e4 d6 d4 Nf6 e5 dxe5 Be2 exd4 Nf3 Nc6 O-O e5 Bg5   
20049  e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...   
20051     e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5   
20055    d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd2   

                                                  move14  \
2      e2e4 e7e5 d2d3 d7d6 c1e3 c7c6 f1e2 b7b5 b1d2 a...   
3      d2d4 d7d5 g1f3 c8f5 b1c3 g8f6 c1f4 f6g4 e2e3 b...   
25     d2d4 d7d5 b1c3 g8f6 c1f4 c8f5 g1f3 e7e6 e2e3 f...   
27     d2d4 d7d5 b1c3 g8f6 g1f3 b8c6 e